<a href="https://colab.research.google.com/github/detektor777/colab_list/blob/main/EZ_DAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[google drive](https://drive.google.com/drive/my-drive)

In [ ]:
INPUT_FILEPATH = "input.mp4" #@param{type:"string"}

OUTPUT_FILE_PATH = "output.mp4" #@param{type:"string"}

TARGET_FPS = 60 #@param{type:"number"}

FRAME_INPUT_DIR = '/content/gdrive/MyDrive/dain_input' #@param{type:"string"}

FRAME_OUTPUT_DIR = '/content/gdrive/MyDrive/dain_output' #@param{type:"string"}

START_FRAME = 1 #@param{type:"number"}

END_FRAME = -1 #@param{type:"number"}

SEAMLESS = False #@param{type:"boolean"}

AUTO_REMOVE = False #@param{type:"boolean"}

In [ ]:
#@title Connect Google Drive
from google.colab import drive
drive.mount('/content/gdrive')
print('Google Drive connected.')

In [ ]:
#@title Check your current GPU
# If you are lucky, you get 16GB VRAM. If you are not lucky, you get less. VRAM is important. The more VRAM, the higher the maximum resolution will go.

# 16GB: Can handle 720p. 1080p will procude an out-of-memory error. 
# 8GB: Can handle 480p. 720p will produce an out-of-memory error.

!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

In [ ]:
#@title Setup everything. This takes a while. Just wait ~20 minutes in total.
%%capture
# Install old pytorch to avoid faulty output
%cd /content/
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
!conda install pytorch==1.1 cudatoolkit torchvision -c pytorch -y
!conda install ipykernel -y

!pip install scipy==1.1.0
!pip install imageio
!CUDA_VISIBLE_DEVICES=0
!sudo apt-get install imagemagick imagemagick-doc
print("Finished installing dependencies.")

# Clone DAIN sources
%cd /content
!git clone -b master --depth 1 https://github.com/baowenbo/DAIN /content/DAIN
%cd /content/DAIN
!git log -1

# Building DAIN
%cd /content/DAIN/my_package/
!./build.sh
print("Building #1 done.")

# Building DAIN PyTorch correlation package.
%cd /content/DAIN/PWCNet/correlation_package_pytorch1_0
!./build.sh
print("Building #2 done.")

# Downloading pre-trained model
%cd /content/DAIN
!mkdir model_weights
!wget -O model_weights/best.pth http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth

In [ ]:
#@title Detecting FPS of input file.
%shell yes | cp -f /content/gdrive/My\ Drive/{INPUT_FILEPATH} /content/DAIN/

import os
filename = os.path.basename(INPUT_FILEPATH)

import cv2
cap = cv2.VideoCapture(f'/content/DAIN/{filename}')

fps = cap.get(cv2.CAP_PROP_FPS)
print(f"Input file has {fps} fps")

if(fps/TARGET_FPS>0.5):
  print("Define a higher fps, because there is not enough time for new frames. (Old FPS)/(New FPS) should be lower than 0.5. Interpolation will fail if you try.")

In [ ]:
#@title ffmpeg extract - Generating individual frame PNGs from the source file.
import os
import subprocess
import re
import time
from tqdm import tqdm

if not os.path.isdir(FRAME_INPUT_DIR):
  os.mkdir(FRAME_INPUT_DIR)

delay = "0.01" #@param [0, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]

cmd = f"ffprobe -v error -select_streams v:0 -show_entries stream=nb_frames -of default=nokey=1:noprint_wrappers=1 '{filename}'"
result = subprocess.run(cmd, stdout=subprocess.PIPE, shell=True, encoding='utf-8')

num_frames = int(re.findall(r'\d+', result.stdout)[0])

start_frame = 0
end_frame = num_frames - 1

for i in tqdm(range(start_frame, end_frame + 1)):
    output_file = f"{FRAME_INPUT_DIR}/{i:05d}.png"
    if os.path.exists(output_file):
        continue
    cmd = f"ffmpeg -i '/content/DAIN/{filename}' -vf 'select=gte(n\,{i}),setpts=PTS-STARTPTS' -vframes 1 {output_file}"
    subprocess.run(cmd, shell=True)
    time.sleep(float(delay))
    
from IPython.display import clear_output
clear_output()

png_generated_count_command_result = %shell ls '{FRAME_INPUT_DIR}' | wc -l
frame_count = int(png_generated_count_command_result.output.strip())

import shutil
if SEAMLESS:
  frame_count += 1
  first_frame = f"{FRAME_INPUT_DIR}/00001.png"
  new_last_frame = f"{FRAME_INPUT_DIR}/{frame_count.zfill(5)}.png"
  shutil.copyfile(first_frame, new_last_frame)

print(f"{frame_count} frame PNGs generated.")

#Checking if PNGs do have alpha
import subprocess as sp
%cd {FRAME_INPUT_DIR}
channels = sp.getoutput('identify -format %[channels] 00001.png')
print (f"{channels} detected")

# Removing alpha if detected
if "a" in channels:
  print("Alpha channel detected and will be removed.")
  print(sp.getoutput('find . -name "*.png" -exec convert "{}" -alpha off PNG24:"{}" \;'))

In [ ]:
#@title Continue

import glob
import re

# Ищем все файлы с расширением .png
png_files = glob.glob(FRAME_OUTPUT_DIR+'/'+'*.png')

if len(png_files) > 0:
    # Если есть файлы, выбираем файл с максимальным номером
    latest_file = max(png_files, key=lambda f: int(re.findall('\d+', f)[-1]))
    print(latest_file)
    # Извлекаем номер из имени файла и делим его на 1000, чтобы получить количество тысяч
    thousands = int(re.findall('\d+', latest_file)[-1]) // 1000
    thousands = thousands-1
else:
    # Если файлы отсутствуют, устанавливаем значение thousands в 1
    thousands = 1

print(f"thousands: {thousands}")


In [ ]:
#@title Interpolation
%shell mkdir -p '{FRAME_OUTPUT_DIR}'
%cd /content/DAIN


!python -W ignore colab_interpolate.py --netName DAIN_slowmotion --time_step {fps/TARGET_FPS} --start_frame {thousands} --end_frame {frame_count} --frame_input_dir '{FRAME_INPUT_DIR}' --frame_output_dir '{FRAME_OUTPUT_DIR}'

In [ ]:
#@title ##**Rename** { display-mode: "form" }
import glob
import os
import time

png_files = sorted(glob.glob(FRAME_OUTPUT_DIR+'/'+'*.png'))
for i, file in enumerate(png_files):
    new_filename = '{:05d}.png'.format(i+1)
    success = False
    while not success:
        try:
            os.rename(file, os.path.join(os.path.dirname(file), new_filename))
            success = True
        except Exception as e:
            print(f"Error renaming {file} to {new_filename}: {e}")
            time.sleep(1)  # ждем 1 секунду перед повторной попыткой

In [ ]:
#@title Create output video
%cd {FRAME_OUTPUT_DIR}
%shell ffmpeg -y -r {TARGET_FPS} -f image2 -pattern_type glob -i '*.png' '/content/gdrive/My Drive/{OUTPUT_FILE_PATH}'





In [ ]:
#@title [Experimental] Create video with sound
# Only run this, if the original had sound.
%cd {FRAME_OUTPUT_DIR}
%shell ffmpeg -i '/content/DAIN/{filename}' -acodec copy output-audio.aac
%shell ffmpeg -y -r {TARGET_FPS} -f image2 -pattern_type glob -i '*.png' -i output-audio.aac -shortest '/content/gdrive/My Drive/{OUTPUT_FILE_PATH}'

if (AUTO_REMOVE):
  !rm -rf {FRAME_OUTPUT_DIR}/*
  !rm -rf output-audio.aac